# PFP Aula 13
## DPEE805 - Modelagem e Controle de Conversores
### Prof° Cassiano Rech | Acadêmico: Tiarles da Rocha Moralles Guterres

Os módulos utilizados neste notebook e importados na célula abaixo são:
* **matplotlib.pyplot**: Para a geração de figuras a partir de vetores.
* **numpy**: É a biblioteca numérica de Python, onde estão implementadas as estruturas e operações com vetores/matrizes.
* **control**: Biblioteca com funções semelhantes as do MATLAB para manipular sistemas utilizando diagrama de blocos.

In [1]:
import control as ctrl
import matplotlib.pyplot as plt
import numpy as np

%matplotlib qt

In [2]:
def fz_eq(fc_desired, theta):
    num = 1 - np.sin(theta)
    den = 1 + np.sin(theta)

    return fc_desired * np.sqrt(num/den)


def fl_eq(fc_desired, theta):
    num = 1 + np.sin(theta)
    den = 1 - np.sin(theta)

    return fc_desired * np.sqrt(num/den)


def w_eq(f):
    return 2*np.pi*f


def deg_to_radians(deg):
    return deg/180*np.pi


def radians_to_deg(radius):
    return radius/np.pi*180

In [26]:
L = 650e-6
C = 680e-6
R = 213.4
Vm = 5
Hi = 1/10
Hv = 1/100
# Hca = 1/311

fs = 70e3
Po = 1500
Vo = 400
fc = fs/10
theta = deg_to_radians(70)

## Malha de corrente (Malha Rápida)

In [27]:
s = ctrl.tf([1, 0], [1]); print(s)


s
-
1



In [28]:
He = s**2/(np.pi*fs)**2 - s/(2*fs) + 1
Gid = Vo/(L*s); print(Gid)

T1 = Gid * Hi/Vm * He


   400
---------
0.00065 s



In [29]:
plt.clf()
_,_,_ = ctrl.bode(T1, Hz=True, dB=True, margins=True, omega_num=10000, omega_limits=(1, 1e6))#, Plot=False)

c:\users\tiarl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
  """Entry point for launching an IPython kernel.


### Escolha de projeto (Malha Rápida/Corrente): Compensador PI

Por ser desejável um ganho em baixas frequência e uma frequência de cruzamento relativamente alta devido ao efeito de comutação que experimenta a corrente de entrada do indutor.

In [30]:
fz = fz_eq(fc, theta); print('fz ', fz)
pc = fs/2

wl = 2*np.pi*fz; print('wl ', wl)
wc = 2*np.pi*pc; print('wc ', wc)

fz  1234.2888649592555
wl  7755.265661127362
wc  219911.4857512855


In [31]:
Ci = (s + wl)/(s * (s + wc)); print(Ci)


    s + 7755
-----------------
s^2 + 2.199e+05 s



In [32]:
ctrl.sisotool(Ci * T1, Hz=True, dB=True, omega_num=10000, omega_limits=(1, 1e6))

Clicked at -1.495e+04+1.225e+04j gain  3.875e+05 damp     0.7737
Clicked at -1.229e+04     +7769j gain  3.636e+05 damp     0.8453
Clicked at      -7858     +9262j gain  2.591e+05 damp     0.6469
Clicked at -1.673e+04     +3291j gain  4.462e+05 damp     0.9812


In [44]:
Ci_adjus = 7.9e5*Ci
plt.clf()
mag, phase, omega = ctrl.bode(Ci_adjus * T1, Hz=True, dB=True, margins=True, omega_num=10000, omega_limits=(1, 1e6))#, Plot=False)

c:\users\tiarl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
  


In [64]:
print(Ci_adjus)


7.9e+05 s + 6.127e+09
---------------------
  s^2 + 2.199e+05 s



In [45]:
Vp = 311
Kv = 1 * 2/np.pi

Gvi = np.pi/4 * Vp/Vo * R/(R*C*s + 1); print(Gvi)


    130.3
------------
0.1451 s + 1



In [46]:
ctrl.pole(Gvi)

array([-6.89122884])

In [54]:
T2 = Kv * 1/Hi * Gvi * Hv

In [49]:
plt.clf()
_,_,_ = ctrl.bode(T2, Hz=True, dB=True, margins=True, omega_num=10000, omega_limits=(1e-1, 1e5))#, Plot=False)

c:\users\tiarl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
  """Entry point for launching an IPython kernel.


## Escolha do compensador (malha lenta/tensão) : Controlador PI

A necessidade neste caso não envolvo o avanço de margim de fase, porém necessita a adição de um zero

In [53]:
fz = 6.81; print('fz ', fz)
wz = w_eq(fz); print('wz ', wz) 

Cv = 1 * (s + wz)/s; print(Cv)

fz  6.81
wz  42.78849194189298

s + 42.79
---------
    s



In [56]:
plt.clf()
ctrl.sisotool(T2*Cv, Hz=True, dB=True, omega_num=10000, omega_limits=(1e-1, 1e2))#, Plot=False)

Clicked at     -50.32    +39.71j gain      1.632 damp      0.785
Clicked at     -46.37    +38.23j gain      1.504 damp     0.7716
Clicked at     -34.52    +39.34j gain      1.094 damp     0.6596
Clicked at      -44.4    +39.34j gain      1.432 damp     0.7484


In [62]:
Cv_adjus = 1.15 * Cv
plt.clf()
_,_,_ = ctrl.bode(Cv_adjus*T2, Hz=True, dB=True, margins=True, omega_num=10000, omega_limits=(1e-1, 1e5))#, Plot=False)

c:\users\tiarl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
  


In [63]:
print(Cv_adjus)


1.15 s + 49.21
--------------
      s

